# Results - smMIPs Sequencing and Data Analysis

## Tools

In [1]:
#!/usr/bin/env python3
import json
import numpy as np
import requests
import sys
import pandas as pd
import seaborn as sns
sns.set(style='white')
sns.set_context("talk")
import matplotlib.pyplot as plt
import glob

In [2]:
from pyliftover import LiftOver
lo = LiftOver('hg19', 'hg38')
li = LiftOver('hg38', 'hg19')

## Pull in Input Files

In [3]:
samples = pd.read_csv('../data/validation_samples/sample_dataframe.txt', sep='\t')

In [4]:
samples_QC = pd.read_csv('../data/validation_samples/sequencing_quality_check.txt', sep='\t')

In [5]:
overlap_with_smmips = pd.read_csv('../output/supplementary_table_3-variant_overlap.tsv', sep='\t')

## Initial Quality Check

In [6]:
print('Mean: ', samples_QC['total tags weighted'].mean())
mean = samples_QC['total tags weighted'].mean()
print('Standard Deviation: ', samples_QC['total tags weighted'].std())
std = samples_QC['total tags weighted'].std()

Mean:  5369817.166666667
Standard Deviation:  3334506.223398502


In [7]:
ineligible_samples = []
for i,row in samples_QC.iterrows():
    if row['total tags weighted'] < mean - 1*std:
        print(row['count'] + '_' +  row['Type'])
        ineligible_samples.append(row['count'] + '_' +  row['Type'])
    

H_ML-08-0075-248_normal
Merck016_tumor
Merck012_tumor
HOC71_normal
SCLC19_normal
SCLC31_normal


In [8]:
print('Number of elgible Samples: ', len(samples_QC) - len(ineligible_samples))

Number of elgible Samples:  30


In [9]:
# If tumor is ineligible, eliminate variants for subsequent analysis
overlap_with_smmips['Passed QC'] = ''
for i,row in overlap_with_smmips.iterrows():
    if row['sample'] + '_' +  'tumor' in ineligible_samples:
        overlap_with_smmips.loc[[i], 'Passed QC'] = 'no'
    else:
        overlap_with_smmips.loc[[i], 'Passed QC'] = 'yes'

In [10]:
print('Number of Eligible Individuals: ', len(overlap_with_smmips[overlap_with_smmips['Passed QC'] == 'yes'][['sample']].drop_duplicates()))

Number of Eligible Individuals:  17


In [11]:
print('Number of Eligible Variants: ',overlap_with_smmips[overlap_with_smmips['Passed QC'] == 'yes'].groupby('sample').size().sum())

Number of Eligible Variants:  74


## Accuracy profile of smMIPs CIViC panel when compared to WEX

 ### Pull in VCF Files

In [12]:
smmips_variants = pd.DataFrame()
for item in glob.glob('../data/smmips_sequencing/*T*.vcf'):
    if item.split('/')[3].startswith('H_'):
        name = item.split('/')[3].split('_')[0] +  '_' +item.split('/')[3].split('_')[1]
    else:
        name = item.split('/')[3].split('_')[0]
    current = pd.read_csv(item, sep='\t', comment='#', header=None).filter(items=[0,1,1,3,4,9])
    current['sample'] = name
    smmips_variants = smmips_variants.append(current)

In [13]:
smmips_variants.columns = ['chrom', 'start', 'stop', 'reference', 'variant', 'CIViC Panel VAF', 'sample']

In [14]:
for i,row in smmips_variants.iterrows():
    VAF = float(row['CIViC Panel VAF'].split(':')[-1])*100
    smmips_variants.loc[[i], 'CIViC Panel VAF'] = VAF

In [15]:
overlap_with_smmips = overlap_with_smmips.merge(smmips_variants, on=['chrom', 'start', 'stop','reference', 'variant', 'sample'], how='left')

In [16]:
overlap_with_smmips = overlap_with_smmips.drop(['Unnamed: 0'], axis=1)
overlap_with_smmips['CIViC Panel VAF'] = overlap_with_smmips['CIViC Panel VAF'].replace(np.nan, 0)
overlap_with_smmips['CIViC Panel VAF'] = overlap_with_smmips['CIViC Panel VAF'].astype('float')

In [23]:
overlap_with_smmips.columns.values

array(['chrom', 'start', 'stop', 'reference', 'variant', 'sample', 'type',
       'gene_name', 'genome', 'strand', 'trv_type', 'c_position',
       'amino_acid', 'tier', 'NORMAL_EXOME_ref_count',
       'NORMAL_EXOME_var_count', 'NORMAL_EXOME_VAF',
       'TUMOR_EXOME_ref_count', 'TUMOR_EXOME_var_count', 'Coverage', 'VAF',
       'Passed QC', 'CIViC Panel VAF'], dtype=object)

In [28]:
print('Total eligible variants: ', len(overlap_with_smmips[overlap_with_smmips['Passed QC'] == 'yes']))
print('Total overlap with smMIPs: ', len(overlap_with_smmips[overlap_with_smmips['CIViC Panel VAF'] > 0]))
print('Non-overlapping variants: ', len(overlap_with_smmips[overlap_with_smmips['Passed QC'] == 'yes']) - len(overlap_with_smmips[overlap_with_smmips['CIViC Panel VAF'] > 0]))


Total eligible variants:  74
Total overlap with smMIPs:  52
Non-overlapping variants:  22


In [29]:
overlap_with_smmips[(overlap_with_smmips['Passed QC'] == 'yes') & (overlap_with_smmips['CIViC Panel VAF'] == 0)]

,chrom,start,stop,reference,variant,sample,type,gene_name,genome,strand,...,tier,NORMAL_EXOME_ref_count,NORMAL_EXOME_var_count,NORMAL_EXOME_VAF,TUMOR_EXOME_ref_count,TUMOR_EXOME_var_count,Coverage,VAF,Passed QC,CIViC Panel VAF
8,chr11,108196129,108196129,C,A,H_ML-08-0075-513,SNP,ATM,37,1.0,...,tier1,546.0,1.0,0.18,377,18,942,4.550000,yes,0.0
9,chr12,25380347,25380347,C,A,H_ML-08-0075-513,SNP,KRAS,37,-1.0,...,tier1,597.0,0.0,0.00,496,26,1119,4.980000,yes,0.0
10,chr13,32913614,32913614,G,A,H_ML-08-0075-513,SNP,BRCA2,37,1.0,...,tier1,349.0,0.0,0.00,273,20,642,6.830000,yes,0.0
11,chr17,41209104,41209104,C,A,H_ML-08-0075-513,SNP,BRCA1,37,-1.0,...,tier1,469.0,0.0,0.00,478,41,988,7.900000,yes,0.0
14,chr1,11187201,11187201,C,T,H_ML-08-0075-513,SNP,MTOR,37,-1.0,...,tier1,764.0,1.0,0.13,866,40,1671,4.420000,yes,0.0
15,chr1,11856310,11856310,C,T,H_ML-08-0075-513,SNP,MTHFR,37,-1.0,...,tier1,627.0,0.0,0.00,660,50,1337,7.040000,yes,0.0
18,chr2,198274698,198274698,C,T,H_ML-08-0075-513,SNP,SF3B1,37,-1.0,...,tier1,611.0,0.0,0.00,580,63,1254,9.800000,yes,0.0
34,chr7,140453161,140453161,T,-,SCLC19,DEL,BRAF,37,-1.0,...,tier1,102.0,0.0,0.00,52,21,175,28.770000,yes,0.0
38,chr17,7578458,7578458,G,-,SCLC19,DEL,TP53,37,-1.0,...,tier1,49.0,0.0,0.00,8,33,90,80.490000,yes,0.0
39,chr10,89690805,89690805,G,A,SCLC25,SNP,PTEN,37,1.0,...,tier1,106.0,0.0,0.00,3,49,158,94.230000,yes,0.0


### Build Waterfall Dataframe 

In [17]:
samples_waterfall = overlap_with_smmips[overlap_with_smmips['Passed QC'] == 'yes']

In [18]:
# Make dataframe for WaterFall plot (Figure 1)
samples_waterfall = samples_waterfall.filter(items=['sample', 'gene_name', 'amino_acid', 'VAF', 'CIViC Panel VAF'])
samples_waterfall['Validated'] = (samples_waterfall['CIViC Panel VAF'] > 0).astype('int')

samples_waterfall.to_csv('../data/validation_samples/waterfall_dataframe.tsv', sep='\t')

## Variant allele frequency correlation between  smMIPs CIViC panel and exome/genome sequencing

In [ ]:
pd.DataFrame.corr(overlap_with_smmips.filter(items=['CIViC Panel VAF', 'VAF']),method='pearson')

In [ ]:
fig, ax = plt.subplots( nrows=1, ncols=1 )
plt.figure(figsize=(5,5))
sns.scatterplot(x="Exome/Genome VAF", y="CIViC Panel VAF", data=samples, alpha=0.8, hue='Tumor Type', palette="deep")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig('../data/Figures/VAF_correlation_TumorType.png', bbox_inches='tight', dpi=400)
plt.close()

In [ ]:
fig, ax = plt.subplots( nrows=1, ncols=1 )
plt.figure(figsize=(5,5))
sns.scatterplot(x="Exome/Genome VAF", y="CIViC Panel VAF", data=samples, alpha=0.8, hue='Matched Normal', palette="deep")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig('../data/Figures/VAF_correlation_MatchedNormal.png', bbox_inches='tight', dpi=400)
plt.close()

In [ ]:
fig, ax = plt.subplots( nrows=1, ncols=1 )
plt.figure(figsize=(5,5))
sns.scatterplot(x="Exome/Genome VAF", y="CIViC Panel VAF", data=samples, alpha=0.8, hue='Passed QC', palette="deep")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig('../data/Figures/VAF_correlation_PassedQC.png', bbox_inches='tight', dpi=400)
plt.close()

In [ ]:
fig, ax = plt.subplots( nrows=1, ncols=1 )
plt.figure(figsize=(5,5))
sns.scatterplot(x="Exome/Genome VAF", y="CIViC Panel VAF", data=samples, alpha=0.8, hue='Status', palette="deep")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig('../data/Figures/VAF_correlation_Status.png', bbox_inches='tight', dpi=400)
plt.close()